## 1. NLTK installation
 1. Install NLTK package using: pip install nltk 
 2. Open python and type the following comands below. Select "all packages" to install data included in NLTK, including corpora, and books. It may take a few minutes to download all data

In [ ]:
import nltk
#nltk.download()

## 2. NLP Objectives and Basic Steps

 - Objectives:
   * Split documents into words, punctuation sysmbols, or segments
   * Understand vocabulary of the text
   * Extract features for further text mining tasks
 - Basic processing steps:
   * Tokenization: split documents into individual words and punctuation symbols
   * Remove stop words and filter tokens
   * POS (part of speech) Tagging
   * Normalization: Stemming, Lemmatization
   * Named Entity Recognition (NER)
   * Term Frequency and Inverse Dcoument Frequency (TF-IDF)
   * Create document-to-term matrix (bag of words)

In [ ]:
import nltk
news=["Oil prices soar to all-time record", 
"Stocks end up near year end", 
"Money funds rose in latest week",
"Stocks up; traders eye crude oil prices",
"Dollar rising broadly on record trade gain"]
text=". ".join(news).lower()
print text

## 3. Tokenization
 - **Definition**: the process of breaking a stream of textual content up into words, terms, symbols, or some other meaningful elements called tokens.
    * Word (Unigram)
    * Sentence
    * Bigram (Two consecutive words)
    * Trigram (Three consecutive words)
   
 - Different methods exist:
    * Split by regular expression patterns
    * NLTK's word tokenizer
    * NLTK's regular expression tokenizer (customizable)
 - None of them can be perfect for any tokenization task. 

### 3.2. Sentence

In [ ]:
# Exercise 3.2.1. Segmentation by Sentences

sentences = nltk.sent_tokenize(text)
print len(sentences)
print sentences

# what patterns can be used to segment 
# text into sentences?

### 3.3 Bigrams (2 consecutive words),  Trigrams (3 consecutive words), or in general n-grams
 - Why bigrams and trigrams?
 - How to get bigrams or trigrams:
    1. First tokenize text into unigrams
    2. Slice through the list of unigrams to get bigrams

In [ ]:
# Exercise 3.3.1. Get bigrams from the text

# unigram tokenization pattern
pattern=r'\w+[\-\.]*\w+'                          

# get unigrams
tokens=nltk.regexp_tokenize(text, pattern)

# bigrams are formed from unigrams
# nltk.bigram returns an iterator
bigrams=list(nltk.bigrams(tokens))
print(bigrams)


 ### 3.4. Collocation
 - Most bigrams or trigrams may sound odd. However, we need to pay attention to frequent bigrams or trigrams
 - **Collocation**: A collocation is a sequence of words that occur together unusually often, e.g. red wine, United States, graduate students, go to school etc.
 - Question: how to find collocations?
   - reference: https://nlp.stanford.edu/fsnlp/promo/colloc.pdf

In [ ]:
# Exercise 3.4.1. Get collocation

from nltk.collocations import *

# bigram association measures
bigram_measures = nltk.collocations.BigramAssocMeasures()

# construct bigrams using words from our example
finder = BigramCollocationFinder.from_words(\
         nltk.regexp_tokenize(text, pattern))

# find the top 10 bigrams by PMI 
# (Pointwise Mutual Information)
# PMI(w1,w2)=log{p(w1,w2)/[p(w1)*p(w2)]}
# due to small corpus, collocations may not be good
finder.nbest(bigram_measures.pmi, 10)  

# construct bigrams using words from a NLTK corpus
#finder = BigramCollocationFinder.from_words(\
#        nltk.corpus.genesis.words('english-web.txt'))
#finder.nbest(bigram_measures.pmi, 10)  
#finder.nbest(bigram_measures.raw_freq, 10) 

## 4. POS (Part of Speech) Tagging

 - What is POS Tagging:
   * The process of marking up a word in a text as corresponding to a particular part of speech (e.g. nouns, verbs, adjectives, adverbs etc.), based on both **its definition**, as well as its **context** — adjacent and related words in a phrase, sentence, or paragraph. 
 - Why POS Tagging: 
   * **disambiguation**: A word may have different meanings. POS tag is a potential strong signal for word sense disambiguation. For example, "I fish a fish"
   * **Phrase extraction**: Use POS rules to define accepted phrases (or information unit) for indexing and retrieval:
        * Adj + Noun, e.g. nice house
        * Verb + Noun, e.g. play football
   * **Filter tokens**:  some POS have less importance in retrieval, e.g. stopwords such as ‘a’, ‘an’, ‘the’, and other glue words like 'in', 'on', 'of' etc.
   * Find other forms of a word based on POS
        * Noun: plural and singular
        * Verb: past, present and future tense
        * Adjective: positive, comparative, and superlative
 - List of Penn Treebank Tags can be found at https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
 - A tagger (program for tagging) is trained based on a corpus using machine learning approaches. It may not be very accurate when applying it your corpus.
   - Stanford tagger (~97%)
   - NLTK default tagger (PerceptronTagger)

In [ ]:
# Exercise 4.1. To find all tags in treebank
nltk.help.upenn_tagset()

# find the meaning of a specific tag
nltk.help.upenn_tagset('JJ')


In [ ]:
# Exercise 4.2. NLTK POS Tagging

# The input to the tagging function is a list of words

# tokenize the text
tokens=nltk.word_tokenize(text)

# tag each tokenized word
tagged_tokens= nltk.pos_tag(tokens)

print tagged_tokens
 

In [ ]:
# Exercise 4.3. Extract Phrases by POS

# Extract phrases in pattern (adjective noun)


phrases=[ [tagged_tokens[i], tagged_tokens[i+1]] \
         for i in range(len(tagged_tokens)-1) \
         if tagged_tokens[i][1].startswith('JJ') and \
         # Tags for nouns can be NN, NNS, NNP, NNPS
         tagged_tokens[i+1][1].startswith ('NN')]

print(phrases)

# how to extract phrase "Noun + Verb"?

## 5. Normalization: Stemming & Lemmatization
 - What is normalization:
   - Converts a list of words in **different surface forms** to a more **uniform form**, e.g.
        * a word with different forms, e.g. organize, organizes, organized, and organizing
        * families of derivationally related words with similar meanings, such as democracy, democratic, and democratization.
 - Why normalization
   - **improve text matching**: in many situations, it seems as if it would be useful for a search for one of these words to return documents that contain another word in the set.
   - reduce featue space generated from text
 - Stemming and lemmatization are two common techinques


### 5.1. Stemming 

* **Stemming**: reducing inflected (or sometimes derived) words to their **stem, base or root** form. 
   * For example, **crying** -> **cri**. 
   * Stemming may not generate a real word, but a root form. 
   * The stemming program is called stemmer. 
       * Famous stemers are Porter stemmer, Lancaster Stemmer, Snowball Stemmer.

In [ ]:
# Exercise 5.1.1. Stermming Using Porter Stemmer

from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

print("Stem of organizing/organized/organizes/organization")
print(porter_stemmer.stem('organizing'))
print(porter_stemmer.stem('organized'))
print(porter_stemmer.stem('organizes'))
print(porter_stemmer.stem('organization'))

print("\nStem of crying")
print(porter_stemmer.stem('crying'))

### 5.2. Lemmatization

* **Lemmatization**: determining the lemma for a given word, 
   * A lemma is a word which stands at the head of a definition in a dictionary, e.g. run (lemma),  runs, ran and running (inflections) 
   * Lemmatization is a complex task involving understanding context and determining the part of speech of a word in a sentence 
      * e.g. "organized" (verb or adjective?)
   * The widely used Lemmatization method is based on WordNet, a large lexical database of English.

* **Difference** between stemming and lemmatization: 
   * a stemmer operates on a single word **without knowledge of the context**, and therefore cannot discriminate between words which have different meanings depending on part of speech. While, lemmatization **requires context and POS tags**. 
   * Stemming may not generate a real word, but lemmization always generates real words.
   *  However, stemmers are typically easier to implement and run faster with reduced accuracy.

In [1]:
# Exercise 5.2.1. Lemmatization

# wordnet lemmatizer takes POS tag as a parameter
# The default POS tag is noun, 

from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

wordnet_lemmatizer = WordNetLemmatizer()

print("organizing (verb)", \
      wordnet_lemmatizer.lemmatize('organizing', wordnet.VERB))
print('organized (verb)', \
      wordnet_lemmatizer.lemmatize('organized', wordnet.VERB))
print('organized (adjective)',\
      wordnet_lemmatizer.lemmatize('organized', wordnet.ADJ))
print('organization (noun)',\
      wordnet_lemmatizer.lemmatize('organization'))
print('crying (adjective)',\
      wordnet_lemmatizer.lemmatize('crying', wordnet.ADJ))
print('crying (verb)', \
      wordnet_lemmatizer.lemmatize('crying', wordnet.VERB))

# compare the result with Exercise 5.1.1.

organizing (verb) organize
organized (verb) organize
organized (adjective) organized
organization (noun) organization
crying (adjective) crying
crying (verb) cry


In [ ]:
# Exercise 5.2.2. Lemmatize the tokens from text 
          
import string
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# first tokenize the text
tokens=nltk.word_tokenize(text)

# then find the POS tag of each word
# tagged_token is a list of (word, pos_tag)
tagged_tokens= nltk.pos_tag(tokens)
print(tagged_tokens)

# wordnet and treebank have different tagging systems
# define a mapping between wordnet tags and POS tags as a function
def get_wordnet_pos(pos_tag):
    
    # if pos tag starts with 'J'
    if pos_tag.startswith('J'):
        # return wordnet tag "ADJ"
        return wordnet.ADJ
    
    # if pos tag starts with 'V'
    elif pos_tag.startswith('V'):
        # return wordnet tag "VERB"
        return wordnet.VERB
    
    # if pos tag starts with 'N'
    elif pos_tag.startswith('N'):
        # return wordnet tag "NOUN"
        return wordnet.NOUN
    
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        # be default, return wordnet tag "NOUN"
        return wordnet.NOUN

# get lemmatized tokens
                            # lemmatize every word in tagged_tokens
le_words=[wordnet_lemmatizer.lemmatize(word, get_wordnet_pos(tag)) \
          # tagged_tokens is a list of tuples (word, tag)
          for (word, tag) in tagged_tokens \
          # remove stop words
          if word not in stop_words and \
          # remove punctuations
          word not in string.punctuation]

# get lemmatized unique tokens as vocabulary
le_vocabulary=set(le_words)

# count each lemmatized tokens
le_word_dict={word: le_words.count(word) \
                    for word in le_words}

print("\nsort dictionary by frequency")
print(sorted(le_word_dict.items(), \
             key=lambda item:-item[1]))

# notice rising/rose -> rise, prices -> price
# notice the two "end" tokens are tagged as VBP and NN perspectively

## 6. Named Entity Recognition (NER)

- Definition: find and classify real word entities (Person, Organization, Event etc.) in text
- Example: sentence "Jim bought 300 shares of Acme Corp. in 2006" can be annotated as "**[Jim]<sub>Person</sub>** bought 300 shares of **[Acme Corp.]<sub>Organization</sub>** in 2006"
- Uses of NER:
   *  Information Extraction: extract clear, factual information, i.e., Who did what to whom when?
   *  Named entities can be indexed, and their relations can be extracted.
   *  Sentiment can be attributed to companies or products
   *  For question answering, answers are often named entities.
- Techniques for NER
   * Regular expression: Telephone numbers, emails, Capital names (e.g. Capitalized word + {city,  center, river}
      * Adantages: simple and sometimes effective
      * Disadvantage: 
         * first word of a sentence is capitalized; sometimes, titles are all capitalized; new proper names constantly emerges (e.g. movie titles, books, etc.)
         * proper names may be ambiguous, e.g. Jordan can be *person* or *location*
   * Supervised learning (IOB) (https://arxiv.org/abs/cmp-lg/9505040)
       1. Collect a set of representative training documents
       2. Label each token for its entity class (I: inside entity, B: begining entity) or other (O)
       3. Design feature extractors appropriate to the text and classes, e.g. current word, pre/next word, pos tags etc.
       4. Train a sequence classifier to predict the labels from the data

In [ ]:
# Exercise 6.1. Use NLTK for Named Entity Recognition

from nltk import word_tokenize, pos_tag, ne_chunk
 
sentence = "Jim bought 300 shares of Acme Corp. in 2006."

# the input to ne_chunk is list of (token, pos tag) tuples
ner_tree=ne_chunk(pos_tag(word_tokenize(sentence)))

# ne_chunk returns a tree
print(ner_tree)

# get PERSON out of the tree
person=[]
for t in ner_tree.subtrees():
    if t.label() == 'PERSON':
        person.append(t.leaves())
print("PERSON",person)

## 7. Term Frequency and Inverse Dcoument Frequency (TF-IDF)
 - Motivation: How to identify important words (or phrases, named entities) in a text in a collecton or corpus? When search for documents, we'd like to have these important words are matched.
 - Intuition: 
   * In a document, if a word/term/phrase is repeated many times, it is likely important. 
   * However, if it appears in most of the documents in the corpus, then it has little discriminating power in determining relevance. 
   * For instance, a collection of documents on the auto industry is likely to have the term auto in almost every document. Search by "auto" you may get all the documents. 
 - ** TF-IDF**: is composed by two terms: 
      - **TF(Term Frequency)**: which measures how frequently a term, say w, occurs in a document. 
      - **IDF (Inverse Document Frequency)**: measures how important a term is within the corpus. 
 
 - TF_IDF provides another way to remove stop words

### 7.1. Term Frequency (TF)
- Measures how frequently a term, say w, occurs in a document, say $d$. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. 
- Thus, the frequency of $w$ in $d$, denoted as $freq(w,d)$ is often divided by the document length (a.k.a. the total number of terms in the document, denoted as $|d|$) as a way of normalization: $$tf(w,d) = \frac{freq(w,d)}{|d|}$$
- Example: d="Stocks end up near year end"
   * tf('Stocks',d)=?
   * tf('end',d)=?

### 7.2. Inverse Document Frequency (IDF)
- Measures how important a term is within the corpus. While computing TF, all terms are considered equally important. 
- However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. 
- Thus we need to weigh down the frequent terms while scale up the rare ones. 
- Let $|D|$ denote the number of documents, $df(w,D)$ denotes the number of documents with term $w$ in them. Then, $$idf(w) = ln(\frac{|D|}{df(w,D)})+1$$ Or a smoothed version: $$idf(t) = ln(\frac{|D|+1}{df(w,D)+1})+1$$
- Examples: 
  * Considering dataset:
       1. "Oil prices soar to all-time record", 
       2. "Stocks end up near year end", 
       3. "Money funds rose in latest week",
       4. "Stocks up; traders eye crude oil prices",
       5. "Dollar rising broadly on record trade gain"
  * idf('Stocks')=?
  * idf('all-time')=?
  * Discussion:
     * What words get very low IDF score?
     * What words get very high IDF score?


### 7.3. TF-IDF 
- Let $s(w,d)=tf(w,d) * idf(w)$, normalize the TF-IDF score of each word in a document normalized by the Euclidean norm, then 
   $$tfidf(w,d)=\frac{s(w,d)}{\sqrt{\sum_{w \in d}{s(w,d)^2}}}$$
- For details of Euclidean norm (a.k.a L2 norm), see https://stanford.edu/class/ee103/lectures/norm/norm_slides.pdf

In [ ]:
# Exercise 7.1. computing tf-idf


import nltk, re, string
from nltk.corpus import stopwords

# library for normalization
from sklearn.preprocessing import normalize

# numpy is the package for matrix caculation
import numpy as np  

stop_words = stopwords.words('english')

docs=["Oil prices soar to all-time record", 
"Stocks end up near year end", 
"Money funds rose in latest week",
"Stocks up; traders eye crude oil prices",
"Dollar rising broadly on record trade gain"]   

In [ ]:
# Step 1. get tokens of each document as list

def get_doc_tokens(doc):
    tokens=[token.strip() \
            for token in nltk.word_tokenize(doc.lower()) \
            if token.strip() not in stop_words and\
               token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, or lemmatization here
    
    return tokens

# step 2. process all documents to get list of token list
docs_tokens=[get_doc_tokens(doc) for doc in docs]

In [ ]:
# step 3. get document-term matrix
# contruct a document-term matrix where 
# each row is a doc 
# each column is a token
# and the value is the frequency of the token

# first get unique tokens of all docs as vocabulary
voc=list(set([token for tokens in docs_tokens \
              for token in tokens]))

# initialize a matrix with all zeros
# the size of the matrix is (len(docs), len(voc))
dtm=np.zeros((len(docs), len(voc)))

# loop through each doc
for row_index,tokens in enumerate(docs_tokens):
    # loop through each token
    for token in tokens:
        # find the column index of the token
        col_index=voc.index(token)
        # increase the cell value by 1
        dtm[row_index, col_index]+=1
        
print(voc)
print(dtm.shape)
print(dtm)

** Term Frequency Matrix **
<img src='tf.png'/>

In [ ]:
# step 4. get normalized term frequency (tf) matrix

# sum the value of each row
doc_len=dtm.sum(axis=1, keepdims=True)
print(doc_len)

# divide dtm matrix by the doc length matrix
tf=np.divide(dtm, doc_len)
print(tf)

In [ ]:
# step 5. get idf

# get a copy of term frequency matrix
# will create idf matrix based on dtm
doc_freq=np.copy(dtm)

# if a term frequency >0, set it to 1, 
# indicating the term appears in the doc
# no matter how many times it appears
doc_freq[np.where(doc_freq>0)]=1
print (doc_freq)

idf=np.log(np.divide(len(docs), np.sum(doc_freq, axis=0)))+1
print("\nIDF Matrix")
print (idf)


smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(doc_freq, axis=0)+1))+1
print("\nSmoothed IDF Matrix")
print(smoothed_idf)


In [ ]:
# step 6. get tf-idf
print("TF-IDF Matrix")
tf_idf=normalize(tf*idf)
print(tf_idf)

print("\nSmoothed TF-IDF Matrix")
smoothed_tf_idf=normalize(tf*smoothed_idf)
print(smoothed_tf_idf)


- TF-IDF matrix gives **weight** of each work in each document
- Documents:
    1. "Oil prices soar to all-time record", 
    2. "Stocks end up near year end", 
    3. "Money funds rose in latest week",
    4. "Stocks up; traders eye crude oil prices",
    5. "Dollar rising broadly on record trade gain"
<img src='tfidf.png'/>

### 7.4. What to do with TF-IDF
- This is the feature sapce of text mining (Bag of Words)
- Find similar documents
    * How to measure simialrity (or distance)? http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.332.4480&rep=rep1&type=pdf
    * Euclidean distance: Euclidean distance is **large** for vectors of high dimension (curse of dimensionality)
    * Cosine similarity: The similarity between two documents is a function of the angle between their vectors in the if-idf vector space. 
      <img src='cosine.png' width=50% />
      <img src='cosine_formula.svg' width=50% />
      - Example: A=[0,2,1], B=[1,1,2], then
      $$cosine(A,B)=\frac{0*1+2*1+1*2}{\sqrt{0+4+1}*\sqrt{1+1+4}}$$
     


In [ ]:
## Exercise 7.4.1 Document similarity

# package to calculate distance
from scipy.spatial import distance

# calculate cosince distance of every pair of documents 
# convert the distance object into a matrix form
# similarity is 1-distance
similarity=1-distance.squareform(distance.pdist(tf_idf_matrix, 'cosine'))
print similarity

# get top docs similar to first doc 
doc_1_sin=similarity[0].tolist()  # convert first row to a list
# sort 
print(sorted(enumerate(doc_1_sim), key=lambda item:-item[1])[0:2])

for idx, doc in enumerate(docs):
    print(idx,doc)

### 7.5. Put Everyting together -- Computing TF-IDF

In [ ]:
import nltk, re, string
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
# numpy is the package for matrix cacluation
import numpy as np  

# Step 1. get tokens of each document as list
def get_doc_tokens(doc):
    stop_words = stopwords.words('english')
    tokens=[token.strip() \
            for token in nltk.word_tokenize(doc.lower()) \
            if token.strip() not in stop_words and\
               token.strip() not in string.punctuation]
    
    # you can add bigrams, collocations, or lemmatization here
    
    return tokens

def tfidf(docs):
    # step 2. process all documents to get list of token list
    docs_tokens=[get_doc_tokens(doc) for doc in docs]
    voc=list(set([token for tokens in docs_tokens \
              for token in tokens]))

    # step 3. get document-term matrix
    dtm=np.zeros((len(docs), len(voc)))

    for row_index,tokens in enumerate(docs_tokens):
        for token in tokens:
            col_index=voc.index(token)
            dtm[row_index, col_index]+=1
            
    # step 4. get normalized term frequency (tf) matrix        
    doc_len=dtm.sum(axis=1, keepdims=True)
    tf=np.divide(dtm, doc_len)
    
    # step 5. get idf
    doc_freq=np.copy(dtm)
    doc_freq[np.where(doc_freq>0)]=1

    smoothed_idf=np.log(np.divide(len(docs)+1, np.sum(doc_freq, axis=0)+1))+1

    
    # step 6. get tf-idf
    smoothed_tf_idf=normalize(tf*smoothed_idf)
    
    return smoothed_tf_idf